## Q.1

In [1]:
from collections import defaultdict

In [2]:
def aprItemFn(transactions, min_support):
    # Convert transactions to a list of sets
    transactions = [set(transaction) for transaction in transactions]
    
    # Calculate the support count for each item
    item_counts = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            item_counts[item] += 1
    
    # Find frequent items based on minimum support
    frequent_items = set()
    item_support = {}
    for item, count in item_counts.items():
        support = count / len(transactions)
        if support >= min_support:
            frequent_items.add(frozenset([item]))
            item_support[frozenset([item])] = support
    
    # Generate frequent itemsets of length k
    k = 2
    while True:
        candidate_sets = aprior_gen(frequent_items, k)
        frequent_items_k = set()
        item_support_k = {}
        
        for candidate in candidate_sets:
            count = sum(1 for transaction in transactions if candidate.issubset(transaction))
            support = count / len(transactions)
            
            if support >= min_support:
                frequent_items_k.add(candidate)
                item_support_k[candidate] = support
        
        if not frequent_items_k:
            break
        
        frequent_items.update(frequent_items_k)
        item_support.update(item_support_k)
        k += 1
    
    return item_support

def aprior_gen(frequent_items, k):
    candidate_sets = []
    for i in frequent_items:
        for j in frequent_items:
            candidate = i.union(j)
            if len(candidate) == k:
                candidate_sets.append(candidate)
    return set(candidate_sets)


In [3]:
# User input for transactions
num_transactions = int(input("Enter the number of transactions: "))
transactions = []
for i in range(num_transactions):
    transaction = input(f"Enter transaction {i+1} : ").split(',')
    transactions.append(transaction)

# User input for minimum support threshold
min_support = float(input("Enter the minimum support threshold (between 0 and 1): "))



Enter the number of transactions: 5
Enter transaction 1 : milk,bread,eggs
Enter transaction 2 : bread,eggs,butter
Enter transaction 3 : milk,eggs,butter
Enter transaction 4 : milk,bread,butter
Enter transaction 5 : milk,bread,cheese
Enter the minimum support threshold (between 0 and 1): 0.3


In [4]:
frequent_itemsets = aprItemFn(transactions, min_support)

for itemset, support in frequent_itemsets.items():
    print(f"Itemset: {set(itemset)}, Support: {support:.2f}")

Itemset: {'milk'}, Support: 0.80
Itemset: {'bread'}, Support: 0.80
Itemset: {'eggs'}, Support: 0.60
Itemset: {'butter'}, Support: 0.60
Itemset: {'eggs', 'butter'}, Support: 0.40
Itemset: {'milk', 'bread'}, Support: 0.60
Itemset: {'bread', 'eggs'}, Support: 0.40
Itemset: {'milk', 'eggs'}, Support: 0.40
Itemset: {'milk', 'butter'}, Support: 0.40
Itemset: {'bread', 'butter'}, Support: 0.40


## Q.2

In [5]:
from collections import defaultdict
from itertools import combinations

In [7]:
def aprRuleFn(frequent_itemsets, min_confidence):
    rules = []
    for itemset, support in frequent_itemsets.items():
        if len(itemset) > 1:
            subsets = [frozenset(subset) for subset in combinations(itemset, len(itemset) - 1)]
            for subset in subsets:
                remaining = itemset - subset
                conf = support / frequent_itemsets[subset]
                if conf >= min_confidence:
                    rules.append((subset, remaining, conf, support))
    return rules
#def aprItemFn(transactions, min_support):
    # ... (function implementation from the previous Code)

In [8]:
# User input for transactions
num_transactions = int(input("Enter the number of transactions: "))
transactions = []
for i in range(num_transactions):
    transaction = input(f"Enter transaction {i+1} : ").split(',')
    transactions.append(transaction)

# User input for minimum support threshold
min_support = float(input("Enter the minimum support threshold (between 0 and 1): "))

min_confidence = float(input("Enter the minimum confidence threshold (between 0 and 1): "))

frequent_itemsets = aprItemFn(transactions, min_support)
rules = aprRuleFn(frequent_itemsets, min_confidence)

Enter the number of transactions: 9
Enter transaction 1 : I1,I2,I5
Enter transaction 2 : I2, I4
Enter transaction 3 : I2, I3
Enter transaction 4 : I1,I2,I4
Enter transaction 5 : I1,I3
Enter transaction 6 : I2,I3
Enter transaction 7 : I1,I3
Enter transaction 8 : I1,I2,I3,I5
Enter transaction 9 : I1,I2,I3
Enter the minimum support threshold (between 0 and 1): 0.3
Enter the minimum confidence threshold (between 0 and 1): 0.5


In [9]:
for rule in rules:
    antecedent, consequent, confidence, support = rule
    print(f"Rule: {set(antecedent)} -> {set(consequent)}, Confidence: {confidence:.2f}, Support: {support:.2f}")

Rule: {'I1'} -> {'I2'}, Confidence: 0.67, Support: 0.44
Rule: {'I2'} -> {'I1'}, Confidence: 0.57, Support: 0.44
Rule: {'I1'} -> {'I3'}, Confidence: 0.67, Support: 0.44
Rule: {'I3'} -> {'I1'}, Confidence: 0.80, Support: 0.44
Rule: {'I3'} -> {'I2'}, Confidence: 0.60, Support: 0.33
